### Primer approach modelo GAN 

__References:__

https://www.kaggle.com/code/amyjang/monet-cyclegan-tutorial

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'